<a href="https://colab.research.google.com/github/Starry5687/pytorch_tutorials/blob/main/quickstart_tutorial_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


[Learn the Basics](intro.html) ||
**Quickstart** ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
[Build Model](buildmodel_tutorial.html) ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Quickstart
This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

## Working with data
PyTorch has two [primitives to work with data](https://pytorch.org/docs/stable/data.html):
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as [TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and [TorchAudio](https://pytorch.org/audio/stable/index.html),
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like
CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:04<00:00, 6508846.09it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 56671.12it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:09<00:00, 474251.69it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 13903591.11it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



datasets.FashionMNIST是一个PyTorch中的数据集，用于训练和测试图像分类模型，它包含了60,000张28x28像素的训练图像和10,000张测试图像，这些图像都属于10个不同的类别，分别是：T-shirt/top、Trouser、Pullover、Dress、Coat、Sandal、Shirt、Sneaker、Bag和Ankle boot。

下面是datasets.FashionMNIST中的参数说明：

root（字符串）：指定数据集的本地目录。
train（布尔值）：如果为True，则加载训练集，否则加载测试集。
download（布尔值）：如果为True，则从Internet下载数据集并将其保存到指定的root目录中。如果数据集已经存在，将不会再次下载。
transform（可调用对象）：对加载的图像应用的变换，ToTensor()是一个内置的PyTorch函数，它将PIL Image或NumPy ndarray转换为张量并标准化像素值。

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element
in the dataloader iterable will return a batch of 64 features and labels.



In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")   # 字典格式打印
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


DataLoader类使得数据加载变得简单，它能够自动将数据分成小批次，并在需要时将它们加载到内存中。在这里，我们使用DataLoader类加载training_data和test_data。在加载完数据后，我们可以通过迭代数据加载器来访问数据，每个迭代都会返回一个由X和y组成的元组，其中X是输入数据，y是标签数据。

在这个例子中，我们只迭代了一个批次的数据，因此在打印出X和y的形状后，我们使用了break语句来停止迭代。print语句用于打印出X和y的形状，以便我们检查数据是否正确地加载到了内存中。最后，我们退出循环并结束了程序。

在分批(batch)处理数据的情况下，每批之后都会调整参数，这样可以提高效率。模型仍然需要对整个训练集进行多轮训练，每轮训练称为一个epoch。每个epoch中，模型将训练集划分为多个batch，依次使用每个batch中的数据进行参数更新。当整个训练集中的数据都被用于训练时，一个epoch就结束了，模型的参数被更新一次。

通常情况下，整个训练集可能会被分成多个batch，而每个batch的数据都是随机的。因此，在一个epoch中，模型将看到训练集中的不同数据，并尝试在训练集上寻找最优的参数配置。

在深度学习中，一般会进行多个epoch的训练，以便进一步优化模型的参数，提高模型的性能。当然，epoch的数量也需要根据具体任务和数据集进行调整，过多的epoch可能会导致模型过拟合，而过少的epoch则可能会导致模型欠拟合。

在深度学习中，通常使用张量（tensor）来表示数据，张量是一个多维数组，每个维度称为一个轴（axis）。

在这段代码中，X是一个张量，它表示一个batch中的输入数据。该指令使用X.shape打印了张量X的形状信息，输出了一个字符串，其中包含了四个数字，分别表示张量的四个轴的大小，这四个数字分别对应于N、C、H、W。

具体来说，这四个参数的含义如下：

N：表示batch中的样本数量（batch size），样本的行数
C：表示输入数据的通道数（channel）   
H：表示输入数据的高度（height）
W：表示输入数据的宽度（width）
因此，这条打印信息的代码指令输出了输入数据的形状信息，其中[N, C, H, W]表示了四个轴的顺序，分别对应于batch size、通道数、高度和宽度。

在计算机科学中，数据的高度和宽度通常是指图像、视频或其他形式的可视化数据的尺寸。

高度通常是指图像或视频的垂直尺寸，即其在垂直方向上的像素数量。通常用像素（pixel）作为度量单位。

宽度通常是指图像或视频的水平尺寸，即其在水平方向上的像素数量。同样地，通常用像素作为度量单位。

例如，一张分辨率为640x480的图像表示该图像在水平方向上有640个像素，在垂直方向上有480个像素。在深度学习中，通常使用高度和宽度来描述输入数据的形状。例如，对于一张图像，其形状可能被表示为（H，W，C），其中H表示图像的高度，W表示图像的宽度，C表示图像的通道数（比如 RGB 彩色图像中的 C 为 3）。


Read more about [loading data in PyTorch](data_tutorial.html).




--------------




## Creating Models
To define a neural network in PyTorch, we create a class that inherits
from [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network
in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate
operations in the neural network, we move it to the GPU or MPS if available.



In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Read more about [building neural networks in PyTorch](buildmodel_tutorial.html).




--------------




## Optimizing the Model Parameters
To train a model, we need a [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions)
and an [optimizer](https://pytorch.org/docs/stable/optim.html).



In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and
backpropagates the prediction error to adjust the model's parameters.



In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.



In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Read more about [Training your model](optimization_tutorial.html).




--------------




## Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

## Loading Models

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it.



In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

This model can now be used to make predictions.



In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Read more about [Saving & Loading your model](saveloadrun_tutorial.html).


